In [47]:
import numpy as np
import pandas as pd
from datetime import datetime

In [283]:
def convert_timestamp(timestamp):
    ts = timestamp.split(" ")
    return datetime.fromisoformat("".join([ts[0], "T", ts[1].split(".")[0], "+00:00"]))
#Testing function
print(convert_timestamp("2025-08-26 04:10:01 +0000 UTC"))

2025-08-26 04:10:01+00:00


In [360]:
#Load in the CSV 
data = pd.read_csv("master.csv")
data["utc_timestamp"] = (data["utc_timestamp"].apply(lambda x: convert_timestamp(x)))
#Need to extract the fullness of the next timestamp
data.sort_values(["utc_timestamp", "name"])
print("Total records: ", len(data))

# Remove any entries such that the time difference is not consistently around 10 mins
garages = ["NorthGarage", "SouthCampusGarage", "SouthGarage", "WestGarage"]
result = []

for garage in garages:
    g = data.loc[data['name'] == garage].copy()
    g['lag1'] = g['fullness'].shift(1)
    g['time_lag1_diff'] = g['utc_timestamp'] - g['utc_timestamp'].shift(1)
    diff = g.loc[(g['time_lag1_diff'] < pd.Timedelta(11, unit="m")) & (g['time_lag1_diff'] > pd.Timedelta(9, unit="m"))]
    result.append(diff)
data = pd.concat(result, ignore_index=True)
data.drop('time_lag1_diff', axis=1)
data['is_weekend'] = data['is_weekend'].astype(int)
data['is_campus_closed'] = data['is_campus_closed'].astype(int)

print("Records with fixed time series: ", len(data))

Total records:  52288
Records with fixed time series:  51072


In [384]:
from sklearn import model_selection

y = data["lag1"]
X = data.drop(columns=["lag1"])
# Split it 75/25 training, testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=10) 
print("training length: ", len(X_train), "testing length: ", len(testing))

training length:  38304 testing length:  12768


In [385]:
from sklearn.preprocessing import OneHotEncoder
# One Hot Encode the parking garage names
def OHE(dataset):
    garage_name = dataset[['name']]
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    enc.fit(garage_name)
    encoded_data = enc.transform(garage_name)
    feature_names = enc.get_feature_names_out(['name'])
    encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=dataset.index)
    return pd.concat([dataset.drop(['name', 'utc_timestamp', "time_lag1_diff", "second"], axis=1), encoded_df], axis=1)
X_train = OHE(X_train)


In [386]:
from sklearn.ensemble import HistGradientBoostingRegressor

boost = HistGradientBoostingRegressor(random_state=10)
model =  boost.fit(X_train, y_train)
model.score(X_train, y_train)

0.9989711576253533

In [387]:
X_test = OHE(X_test)
model.score(X_test, y_test)

0.9993559236273146

In [394]:
sample = X_test.loc[30648]
sample

fullness                    45.0
hour                        20.0
minute                      20.0
year                      2025.0
month                       10.0
day                          1.0
weekday                      3.0
is_weekend                   0.0
is_campus_closed             0.0
name_NorthGarage             0.0
name_SouthCampusGarage       0.0
name_SouthGarage             1.0
name_WestGarage              0.0
Name: 30648, dtype: float64

In [392]:
model.predict([sample])

/Users/ryantan/Desktop/sjsubark/inference/.venv/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but HistGradientBoostingRegressor was fitted with feature names
  warnings.warn(


array([46.73626293])

In [395]:
y_test.loc[30648]

np.float64(47.0)